In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output

In [2]:
%time
path = 'Data/Santander/'
os.listdir(path)

Wall time: 0 ns


['test.csv', 'test.csv.zip', 'train.csv', 'train.csv.zip']

In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import BayesianRidge
warnings.filterwarnings('ignore')

from fastai.tabular import *
from fastai import *


In [3]:
PATH="../input/"

In [4]:
%%time
train_df = pd.read_csv(PATH+"train.csv")
test_df = pd.read_csv(PATH+"test.csv")

CPU times: user 16.8 s, sys: 1.09 s, total: 17.9 s
Wall time: 17.9 s


In [5]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

# Deep Neural Network Tabular:

In [7]:
def augment_df(df):
    for feature in features:
        df[f'sq_{feature}'] = df[feature]**2
        df[f'repo_{feature}'] = df[feature].apply(lambda x: 0 if x==0 else 1/x)
    df['min'] = df[features].min(axis=1)
    df['mean'] = df[features].mean(axis=1)
    df['max'] = df[features].max(axis=1)
    df['median'] = df[features].median(axis=1)
    df['std'] = df[features].std(axis=1)
    df['var'] = df[features].var(axis=1)
    df['abs_mean'] = df[features].abs().mean(axis=1)
    df['abs_median'] = df[features].abs().median(axis=1)
    df['abs_std'] = df[features].abs().std(axis=1)
    df['skew'] = df[features].skew(axis=1)
    df['kurt'] = df[features].kurt(axis=1)
    df['sq_kurt'] = df[[f'sq_{feature}' for feature in features]].kurt(axis=1)

In [8]:
augment_df(train_df)
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,sq_var_186,repo_var_186,sq_var_187,repo_var_187,sq_var_188,repo_var_188,sq_var_189,repo_var_189,sq_var_190,repo_var_190,sq_var_191,repo_var_191,sq_var_192,repo_var_192,sq_var_193,repo_var_193,sq_var_194,repo_var_194,sq_var_195,repo_var_195,sq_var_196,repo_var_196,sq_var_197,repo_var_197,sq_var_198,repo_var_198,sq_var_199,repo_var_199,min,mean,max,median,std,var,abs_mean,abs_median,abs_std,skew,kurt,sq_kurt
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,140.211649,0.084452,388.716713,-0.050720,308.856020,0.056901,0.343044,1.707359,19.672773,0.225459,15.714882,0.252258,9.837005,0.318837,2.859481,0.591366,343.090415,0.053988,5.749445,-0.417049,62.069187,0.126929,73.333532,0.116775,163.336068,0.078245,1.191154,-0.916254,-21.4494,7.281591,43.1127,6.77040,9.331540,87.077642,9.433039,8.41525,7.136966,0.101580,1.331023,23.320706
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,58.588308,0.130646,253.825438,-0.062767,177.355806,0.075089,0.127164,-2.804262,58.401692,0.130854,59.620018,0.129510,6.675506,0.387042,119.937543,0.091311,238.100330,0.064807,4.136749,0.491666,66.043253,0.123051,77.244763,0.113780,336.942736,0.054478,3.809523,0.512348,-47.3797,7.076818,40.5632,7.22315,10.336130,106.835574,9.684721,8.03470,7.931169,-0.351734,4.110215,21.143081
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,28.885250,0.186064,39.262756,-0.159591,103.905404,0.098103,0.708459,-1.188072,8.443092,0.344151,95.853890,0.102140,2.790236,0.598659,2.841922,0.593190,466.741458,0.046287,9.870279,0.318299,42.527354,-0.153344,68.351556,0.120956,216.743173,0.067925,0.157212,2.522068,-22.4038,6.204483,33.8820,5.89940,8.753387,76.621777,8.618451,7.28465,6.376510,-0.056957,0.546438,10.849758
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,62.942009,0.126046,164.555018,-0.077955,154.067674,0.080565,3.418431,0.540862,19.950516,0.223884,22.498895,0.210824,0.515237,1.393146,2.020378,0.703532,530.597404,0.043413,1.614424,-0.787030,8.570256,-0.341588,105.929381,0.097161,322.910118,0.055649,80.992800,-0.111116,-35.1659,6.441159,38.1015,6.70260,9.594064,92.046058,9.095397,7.74480,7.113447,-0.480116,2.630499,14.138277
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,41.227957,0.155741,35.129329,0.168719,256.643604,0.062422,0.080032,-3.534818,2.221590,-0.670916,90.657058,0.105027,0.022741,-6.631300,84.533314,0.108764,176.560314,0.075258,2.286446,-0.661332,15.418973,0.254667,90.308910,0.105229,323.906407,0.055564,77.623148,-0.113502,-65.4863,6.771155

In [9]:
augment_df(test_df)
test_df.head()


,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,sq_var_186,repo_var_186,sq_var_187,repo_var_187,sq_var_188,repo_var_188,sq_var_189,repo_var_189,sq_var_190,repo_var_190,sq_var_191,repo_var_191,sq_var_192,repo_var_192,sq_var_193,repo_var_193,sq_var_194,repo_var_194,sq_var_195,repo_var_195,sq_var_196,repo_var_196,sq_var_197,repo_var_197,sq_var_198,repo_var_198,sq_var_199,repo_var_199,min,mean,max,median,std,var,abs_mean,abs_median,abs_std,skew,kurt,sq_kurt
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,-2.0248,-4.3554,13.9696,0.3458,7.5408,14.5001,7.7028,-19.0919,15.5806,16.1763,3.7088,18.8064,1.5899,3.0654,6.4509,14.1192,-9.4902,-2.1917,5.7107,3.7864,-1.7981,9.2645,2.0657,12.7753,11.3334,8.1462,-0.0610,3.5331,9.7804,...,86.255799,0.107673,555.568470,-0.042426,175.941654,0.075390,2.752613,0.602736,4.646611,-0.463908,140.410650,0.084392,2.044900,-0.699301,6.006421,0.408030,187.997005,0.072933,6.085596,0.405367,19.056717,0.229074,114.918400,0.093284,239.388973,0.064632,76.033168,-0.114683,-31.9891,7.083202,42.0248,7.31440,9.910632,98.220627,9.576049,8.47350,7.515516,-0.088518,1.871262,15.607325
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,-1.3809,-0.3310,14.1129,2.5667,5.4988,14.1853,7.0196,4.6564,29.1609,0.0910,12.1469,3.1389,5.2578,2.4228,16.2064,13.5023,-5.2341,-3.6648,5.7080,2.9965,-10.4720,11.4938,-0.9660,15.3445,10.6361,0.8966,6.7428,2.3421,12.8678,...,139.861372,0.084557,75.885005,-0.114795,253.039012,0.062865,0.962753,1.019160,112.710072,0.094193,78.055458,0.113187,0.884164,1.063490,102.580435,0.098734,242.627352,0.064199,0.227815,2.095118,2.205819,-0.673310,97.444538,0.101303,365.930118,0.052276,439.992576,-0.047674,-41.1924,6.248430,35.6020,6.43960,9.541267,91.035772,8.960003,7.94920,7.042000,-0.559785,3.391068,20.159428
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,-4.7057,-3.0422,13.6751,3.8183,10.8535,14.2126,9.8837,2.6541,21.2181,20.8163,12.4666,12.3696,4.7473,2.7936,5.2189,13.5670,-15.4246,-0.1655,7.2633,3.4310,-9.1508,9.7320,3.1062,22.3076,11.9593,9.9255,4.0702,4.9934,8.0667,...,24.184757,0.203343,4.293184,-0.482625,133.148521,0.086663,1.397360,0.845952,0.560103,-1.336184,120.857042,0.090963,3.921588,0.504974,4.752400,0.458716,168.514150,0.077034,4.528810,0.469903,50.532194,-0.140675,49.869019,0.141607,395.834899,0.050262,537.284584,-0.043142,-34.3488,7.151299,39.3654,7.26355,9.967466,99.350374,9.554517,8.27835,7.681205,-0.135084,2.326901,12.575372
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,0.0095,-5.0659,14.0526,13.5010,8.7660,14.7352,10.0383,-15.3508,2.1273,21.4797,14.5372,12.5527,2.9707,4.2398,13.7796,14.1408,1.0061,-1.3479,5.2570,6.5911,6.2161,9.5540,2.3628,10.2124,10.8047,-2.5588,6.0720,3.2613,16.5632,...,10.957424,0.302097,391.323568,-0.050551,180.899810,0.074350,1.717148,0.763126,91.588728,0.104491,82.384668,0.110173,2.748964,0.603136,12.825710,0.279228,230.657119,0.065844,10.021023,0.315896,15.655475,0.252736,85.183670,0.108348,169.437082,0.076824,17.730837,-0.237485,-21.4797,7.057223,40.3383,6.89675,8.257204,68.181422,8.826497,7.91085,6.319616,-0.167741,2.253054,30.958931
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,5.1025,-3.2827,14.1013,8.9672,4.7276,14.5811,11.8615,3.1480,18.0126,13.8006,1.6026,16.3059,6.7954,3.6015,13.6569,13.8807,8.6228,-2.2654,5.2255,7.0165,-15.6961,10.6239,-4.7674,17.5447,11.8668,3.0154,4.2546,6.7601,5.9613,...,69.948132,0.119567,616.300485,-0.040281,132.084452,0.087011,2.663750,0.612708,17.858231,0.236636,84.131087,0.109024,1.647372,0.779120,11.409533,0.296051,382.366738,0.051140,0.081796,-3.496503,26.637985,-0.193753,53.117859,0.137208,193.933476,0.071808,84.356877,-0.10

In [10]:
features = features + [f'sq_{feature}' for feature in features] + [f'repo_{feature}' for feature in features]
num_features = len(features)

In [11]:
num_features

600

In [12]:
random.seed(2)
valid_idx = random.sample(list(train_df.index.values), int(len(train_df)*0.05))
train_idx = train_df.drop(valid_idx).index

In [13]:
summary = train_df.iloc[train_idx].describe()
summary

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,...,sq_var_186,repo_var_186,sq_var_187,repo_var_187,sq_var_188,repo_var_188,sq_var_189,repo_var_189,sq_var_190,repo_var_190,sq_var_191,repo_var_191,sq_var_192,repo_var_192,sq_var_193,repo_var_193,sq_var_194,repo_var_194,sq_var_195,repo_var_195,sq_var_196,repo_var_196,sq_var_197,repo_var_197,sq_var_198,repo_var_198,sq_var_199,repo_var_199,min,mean,max,median,std,var,abs_mean,abs_median,abs_std,skew,kurt,sq_kurt
count,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,...,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,1.900000e+05,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,1.900000e+05,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000,190000.000000
mean,0.100395,10.680067,-1.630230,10.714336,6.795830,11.078848,-5.063642,5.408206,16.546994,0.280909,7.567228,0.396483,-3.245270,14.024154,8.531778,7.536421,14.573067,9.333755,-5.692841,15.243729,12.435990,13.287559,17.259553,4.303709,3.019969,10.579091,13.667467,-4.053773,-1.137426,5.532722,5.053374,-7.692313,10.393210,-0.511490,14.771887,11.434721,3.842072,2.187240,5.869074,10.639304,...,90.619289,0.134813,240.531816,-0.060219,252.028565,0.070149,1.510224,0.498602,31.257836,0.069201,64.476783,0.167922,5.904059,0.419947,2.700718e+01,0.250004,333.629545,0.057394,2.061722,-0.074256,3.504984e+01,0.113815,80.193152,0.113503,261.020668,0.065624,119.885189,-0.147546,-30.439284,6.767338,39.277405,6.668603,9.662164,93.677213,9.316794,7.894756,7.231617,-0.179733,2.250880,19.756599
std,0.300526,3.040860,4.050303,2.640776,2.043235,1.622706,7.862281,0.866528,3.416396,3.334549,1.234565,5.500587,5.972695,0.190046,4.639418,2.248982,0.411970,2.557058,6.715312,7.850442,7.995298,5.873795,8.196789,2.846936,0.526908,3.779365,0.285528,5.923713,1.523774,0.782696,2.614674,7.963504,2.161989,2.587638,4.322056,0.541634,5.182104,3.118226,2.250207,4.278391,...,59.686714,0.890231,285.486646,19.433113,123.265985,0.021317,1.822448,98.927178,40.390501,31.042690,48.275152,1.822532,6.559641,71.132975,3.235326e+01,43.255155,113.391001,0.010703,2.636260,78.284771,4.099041e+01,33.006560,16.418580,0.012089,94.774649,0.014406,150.693020,37.229430,9.053609,0.401580,5.646011,0.402832,0.565503,10.978650,0.335652,0.467565,0.466829,0.392644,1.455286,12.858450
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.277800,-32.562600,2.347300,5.349700,-10.505500,3.970500,-20.731300,-26.095000,13.488400,-4.987400,1.013300,13.076900,0.635100,-33.380200,-10.664200,-12.402500,-5.432200,-10.089000,-5.322500,1.209800,-0.678400,12.720000,-24.016000,-6.166800,2.798900,-4.787200,-34.798400,2.140600,-8.986100,1.508500,9.816900,-16.513600,-8.095100,-1.183400,-6.337100,...,0.000008,-106.382979,0.000000,-3333.333333,19.468391,0.034094,0.000000,-10000.000000,0.000000,-5000.000000,0.000004,-476.190476,0.000000,-10000.000000,1.000000e-08,-5000.000000,75.592591,0.035805,0.000000,-10000.000000,1.000000e-08,-5000.

In [14]:
train_df.iloc[valid_idx].target.sum() / len(valid_idx), train_df.target.sum()/len(train_df)

(0.1023, 0.10049)

In [15]:
class roc(Callback):
    def on_epoch_begin(self, **kwargs):
        self.total = 0
        self.batch_count = 0
    
    def on_batch_end(self, last_output, last_target, **kwargs):
        preds = F.softmax(last_output, dim=1)
        try: 
            roc_score = roc_auc_score(to_np(last_target), to_np(preds[:, -1]))
            self.total+=roc_score
            self.batch_count+=1
        except:
            pass
    def on_epoch_end(self, num_batch, **kwargs):
        self.metric = self.total/self.batch_count

In [16]:
bs = 2048

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [18]:
def train_and_eval_tabular_learner(train_df, train_features, valid_idx, add_noise=False, lr=0.02, epochs=1, layers=[200, 100], ps=[0.5, 0.2], name='learner'):
    data = TabularDataBunch.from_df(path='.', df=train_df, dep_var='target', valid_idx=valid_idx,
                                    cat_names=[], cont_names=train_features, bs=bs, procs=[FillMissing, Normalize], test_df=test_df)
    learn = tabular_learner(data, layers=layers, ps=ps, metrics=[roc()])
    if add_noise:
        data = None
        noise = np.random.normal(summary[features].loc['mean'].values, summary[features].loc['std'].values, (len(train_df), num_features))/100
        train_df[features]+=noise
        data = TabularDataBunch.from_df(path='.', df=train_df, dep_var='target', valid_idx=valid_idx,
                                    cat_names=[], cont_names=train_features, bs=bs, procs=[FillMissing, Normalize], test_df=test_df)
        learn.data = data
        learn.fit_one_cycle(epochs, lr)
        train_df[features]-=noise
        noise = None
    else:
        learn.fit_one_cycle(epochs, lr)
    learn.save(name, with_opt=False)
    valid_preds, _ = learn.get_preds(ds_type=DatasetType.Valid)
    valid_probs = np.array(valid_preds[:, -1])
    valid_targets = train_df.loc[valid_idx].target.values
    valid_score = roc_auc_score(valid_targets, valid_probs)
    test_preds, _ = learn.get_preds(ds_type=DatasetType.Test)
    test_probs = to_np(test_preds[:, -1])
    return valid_score, valid_probs, test_probs

In [19]:
sub_features = []
valid_scores = []
valid_preds = []
preds = []
num_epochs = 10
cv_counts = len(train_df)//num_epochs
saved_model_prefix='learner'

In [20]:
augmented_features = ['min', 'mean', 'max', 'median', 'std', 'abs_mean', 'abs_median', 'abs_std', 'skew', 'kurt', 'sq_kurt']

In [21]:
for i in range(num_epochs):
    print('Training model: ', i)
    sub_features.append(random.sample(list(features), int(num_features*0.75)) + augmented_features)
    name = f'{saved_model_prefix}_{i}'
    score, valid_probs, test_probs = train_and_eval_tabular_learner(train_df, sub_features[-1], valid_idx, add_noise=True, epochs=3, lr=0.02, name=name)
    valid_scores.append(score)
    valid_preds.append(valid_probs)
    preds.append(test_probs)

In [33]:
x = sorted(np.std(preds, axis = 0))
x

[0.00011796789,
 0.00013750253,
 0.00013773689,
 0.0001407769,
 0.00014427549,
 0.00014649454,
 0.00014684854,
 0.00014877312,
 0.00015439054,
 0.0001575871,
 0.00016255151,
 0.00016519234,
 0.00016610665,
 0.00016634996,
 0.00016665683,
 0.00017230991,
 0.00017423765,
 0.00018141902,
 0.00018407282,
 0.00018806281,
 0.00019024157,
 0.00019504956,
 0.00019631554,
 0.0001985843,
 0.00020331498,
 0.00020710097,
 0.00021125657,
 0.00021242663,
 0.00021621783,
 0.00021971166,
 0.00022191796,
 0.00022528601,
 0.00022784335,
 0.0002303211,
 0.00023183825,
 0.00023295358,
 0.00023337273,
 0.00023883919,
 0.00024057641,
 0.00024095635,
 0.00024116322,
 0.00024350148,
 0.00024362977,
 0.0002450694,
 0.00024785675,
 0.00024891258,
 0.0002513371,
 0.0002571571,
 0.00026042262,
 0.0002606884,
 0.00026112434,
 0.0002621776,
 0.00026263506,
 0.0002650959,
 0.00026573992,
 0.00026729293,
 0.00026757858,
 0.00026776508,
 0.00026798167,
 0.00027409638,
 0.00027425578,
 0.00027604055,
 0.00027634198,
 0

# Gradient Boosting

In [34]:
%%time
train_df = pd.read_csv(PATH+"train.csv")
test_df = pd.read_csv(PATH+"test.csv")

CPU times: user 17.1 s, sys: 2.01 s, total: 19.1 s
Wall time: 19.1 s


In [35]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

## Light GBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
target = train_df['target']

In [ ]:
preds

In [43]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [ ]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

In [ ]:
#kfold = 15
#folds = StratifiedKFold(n_splits=kfold, shuffle=False, random_state=44000)
num_folds = 3
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]



folds = KFold(n_splits=num_folds, random_state=44000)
oof = np.zeros(len(train_df))
getVal = np.zeros(len(train_df))
predictions = np.zeros(len(target))
feature_importance_df = pd.DataFrame()

In [ ]:
print('Light GBM Model')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    
    X_train, y_train = train_df.iloc[trn_idx][features], target.iloc[trn_idx]
    X_valid, y_valid = train_df.iloc[val_idx][features], target.iloc[val_idx]
    
    X_tr, y_tr = augment(X_train.values, y_train.values)
    X_tr = pd.DataFrame(X_tr)
    
    print("Fold idx:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    getVal[val_idx]+= clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration) / folds.n_splits
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

In [ ]:
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

In [ ]:
predictions

## XG Boost

In [46]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score, confusion_matrix
import xgboost as xgb

In [47]:
train_cols = [c for c in train_df.columns if c not in ["ID_code", "target"]]
y_train = train_df["target"]

In [48]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1001)

In [49]:
# bayesian hpo
# details: https://xgboost.readthedocs.io/en/latest/parameter.html
params = {'tree_method': 'hist',
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'learning_rate': 0.0936165921314771,
 'max_depth': 2,
 'colsample_bytree': 0.3561271102144279,
 'subsample': 0.8246604621518232,
 'min_child_weight': 53,
 'gamma': 9.943467991283027,
 'silent': 1}

In [ ]:
%%time

oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])

feature_importance_df = pd.DataFrame()

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_df, y_train)):
    
    trn_x, trn_y = train_df[train_cols].iloc[trn_idx], y_train.iloc[trn_idx]
    val_x, val_y = train_df[train_cols].iloc[val_idx], y_train.iloc[val_idx]
    
    dtrain = xgb.DMatrix(trn_x, trn_y, feature_names=trn_x.columns)
    dval = xgb.DMatrix(val_x, val_y, feature_names=val_x.columns)
    
    clf = xgb.train(params=params, dtrain=dtrain, num_boost_round=4000, evals=[(dtrain, "Train"), (dval, "Val")],
        verbose_eval= 100, early_stopping_rounds=50) 
    
    oof_preds[val_idx] = clf.predict(xgb.DMatrix(val_x))
    sub_preds += clf.predict(xgb.DMatrix(test_df[train_cols])) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].index
    fold_importance_df["fscore"] = pd.DataFrame.from_dict(data=clf.get_fscore(), orient="index", columns=["FScore"])["FScore"].values
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print('\nFold %1d AUC %.6f & std %.6f' %(n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx]), np.std([oof_preds[val_idx]])))
    print('Fold %1d Precision %.6f' %(n_fold + 1, precision_score(val_y, np.round(oof_preds[val_idx])) ))
    print('Fold %1d Recall %.6f' %(n_fold + 1, recall_score(val_y, np.round(oof_preds[val_idx]) )))
    print('Fold %1d F1 score %.6f' % (n_fold + 1,f1_score(val_y, np.round(oof_preds[val_idx]))))
    print('Fold %1d Kappa score %.6f\n' % (n_fold + 1,cohen_kappa_score(val_y, np.round(oof_preds[val_idx]))))
    gc.collect()

print('\nCV AUC score %.6f & std %.6f' % (roc_auc_score(y_train, oof_preds), np.std((oof_preds))))
print('CV Precision score %.6f' % (precision_score(y_train, np.round(oof_preds))))
print('CV Recall score %.6f' % (recall_score(y_train, np.round(oof_preds))))
print('CV F1 score %.6f' % (f1_score(y_train, np.round(oof_preds))))
print('CV Kappa score %.6f' % (cohen_kappa_score(y_train, np.round(oof_preds))))

In [ ]:
preds1 = np.array(sum(preds)/num_epochs)
preds1

In [ ]:
predictions.shape, preds1.shape

In [ ]:
all_ensemble_values = [0., 0.125, 0.25, 0.375, 0.5, 0.675, 0.75, 0.875, 1.0]

In [ ]:
sample_ensemble_array = np.array(all_ensemble_values[0]*preds1 + (1-all_ensemble_values[0])*predictions)
sample_ensemble_array

In [ ]:
for i in range(len(all_ensemble_values)):
    predict_array = np.array(all_ensemble_values[i]*preds1 + (1-all_ensemble_values[i])*predictions)
    sub_df = pd.DataFrame({'ID_code': test_df['ID_code'].values})
    sub_df['target'] = predict_array
    sub_df.to_csv(f'submission_{i}.csv', index=False)

In [ ]:
# from IPython.display import FileLink
# FileLink(f'submit_dl.csv')